# Как быстро посчитать расстояние между парами объектов

$ X $ - n x d матрица 

$ Y $ - m x d матрица

Необходимо найти матрицу $D$, такую что $D[i, j]$ - это расстояние между $i-ой$ строкой матрицы $X$ и $j-ой$ строкой матрицы $Y$ 

$D$ - n x m матрица

$ D[i, j] = dist(X[i, :], Y[j, :]) $ $ \forall i, j$

В качестве расстояния возьмем евклидово.

Пусть $x$ - строка матрицы X, а $y$ - строка матрицы Y, тогда

$dist(x, y) = \sqrt{\sum_{i=0}^d (x_i - y_i)^2}$

In [1]:
import pandas as pd
import numpy as np
from line_profiler import LineProfiler
from sklearn import datasets

%load_ext Cython

In [2]:
def profile_print(func_to_call, *args):
    profiler = LineProfiler()
    profiler.add_function(func_to_call)
    profiler.runcall(func_to_call, *args)
    profiler.print_stats()

In [3]:
num_features = 100

# we will need only feature representations
X = datasets.make_blobs(777, n_features=num_features, centers=10)[0]
Y = datasets.make_blobs(500, n_features=num_features, centers=30)[0]
X.shape, Y.shape

((777, 100), (500, 100))

### Naive way

In [4]:
def vector_distance(x, y):
    dist = ((x - y) ** 2).sum()
    return np.sqrt(dist)


def naive_mdist(X, Y):
    D = np.zeros((X.shape[0], Y.shape[0]))
    for i, x in enumerate(X):
        for j, y in enumerate(Y):
            D[i, j] = vector_distance(x, y)
    return D

In [5]:
%%timeit
naive_mdist(X, Y)

2.78 s ± 299 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [6]:
profile_print(naive_mdist, X, Y)

Timer unit: 1e-06 s

Total time: 3.44746 s
File: <ipython-input-4-bfda368bb7c6>
Function: naive_mdist at line 6

Line #      Hits         Time  Per Hit   % Time  Line Contents
     6                                           def naive_mdist(X, Y):
     7         1          746    746.0      0.0      D = np.zeros((X.shape[0], Y.shape[0]))
     8       778          631      0.8      0.0      for i, x in enumerate(X):
     9    389277       291432      0.7      8.5          for j, y in enumerate(Y):
    10    388500      3154652      8.1     91.5              D[i, j] = vector_distance(x, y)
    11         1            1      1.0      0.0      return D



In [7]:
D_naive = naive_mdist(X, Y)

In [8]:
D_naive.shape

(777, 500)

## Better numpy

### How to write euclidean distance in vector-matrix form ? 

#### Broadcasting
http://scipy.github.io/old-wiki/pages/EricsBroadcastingDoc

In [9]:
broad = X - Y[0]
not_broad = np.array([x - Y[0] for x in X])

print('Shapes:', broad.shape, not_broad.shape)
print('Number of different values:', (broad != not_broad).sum())

Shapes: (777, 100) (777, 100)
Number of different values: 0


In [10]:
%%timeit 
X - Y[0]

94.4 µs ± 1.45 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [11]:
%%timeit
np.array([x - Y[0] for x in X])

1.51 ms ± 32.8 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [12]:
def broadcast_numpy_dist(X, Y):
    D = np.zeros((X.shape[0], Y.shape[0]))
    for j, y in enumerate(Y):
        dist = ((X - y) ** 2).sum(axis=1)
        D[:, j] = np.sqrt(dist)
    return D

In [13]:
D_broadcast = broadcast_numpy_dist(X, Y)
np.abs(D_broadcast - D_naive).sum()

0.0

In [14]:
%%timeit 
broadcast_numpy_dist(X, Y)

99.9 ms ± 749 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [15]:
profile_print(broadcast_numpy_dist, X, Y)

Timer unit: 1e-06 s

Total time: 0.129855 s
File: <ipython-input-12-97aa9aea5a19>
Function: broadcast_numpy_dist at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def broadcast_numpy_dist(X, Y):
     2         1         1036   1036.0      0.8      D = np.zeros((X.shape[0], Y.shape[0]))
     3       501          707      1.4      0.5      for j, y in enumerate(Y):
     4       500       118210    236.4     91.0          dist = ((X - y) ** 2).sum(axis=1)
     5       500         9902     19.8      7.6          D[:, j] = np.sqrt(dist)
     6         1            0      0.0      0.0      return D



### Pure numpy

$x$, $y$ - одномерные вектора
$$ dist(x, y)^2 = \sum_i(x_i - y_i)^2 = \sum_i(x_i^2 - 2x_i y_i + y_i^2) = \sum_ix_i^2 -2 \sum_ix_i y_i + \sum_iy_i^2$$

$$ numpy :  np.sum(x \cdot \cdot 2 - 2 \cdot x \cdot y - y \cdot \cdot 2) $$

In [20]:
def numpy_dist(X, Y):
    x_dist = (X ** 2).sum(axis=1)
    y_dist = (Y ** 2).sum(axis=1)
    xy_dist = X.dot(Y.T)
    dist = - 2 * xy_dist + y_dist + x_dist.reshape(-1, 1)
    return np.sqrt(dist)

In [21]:
D_numpy = numpy_dist(X, Y)
np.abs(D_numpy - D_naive).sum()

1.727713083710114e-09

In [22]:
%%timeit 
numpy_dist(X, Y)

10 ms ± 698 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [23]:
profile_print(numpy_dist, X, Y)

Timer unit: 1e-06 s

Total time: 0.01111 s
File: <ipython-input-20-85a26e63ca8e>
Function: numpy_dist at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def numpy_dist(X, Y):
     2         1          285    285.0      2.6      x_dist = (X ** 2).sum(axis=1)
     3         1          166    166.0      1.5      y_dist = (Y ** 2).sum(axis=1)
     4         1         4540   4540.0     40.9      xy_dist = X.dot(Y.T)
     5         1         4482   4482.0     40.3      dist = - 2 * xy_dist + y_dist + x_dist.reshape(-1, 1)
     6         1         1637   1637.0     14.7      return np.sqrt(dist)



### Library (maybe the best way)

In [24]:
from scipy.spatial.distance import cdist

In [25]:
D_scipy = cdist(X, Y, metric='euclidean')
np.abs(D_scipy - D_naive).sum()

3.3919462794074207e-09

In [26]:
%%timeit 
cdist(X, Y, metric='euclidean')

40.2 ms ± 1.64 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [26]:
profile_print(cdist, X, Y)

Timer unit: 1e-06 s

Total time: 0.044613 s
File: /Users/meudon/anaconda3/lib/python3.6/site-packages/scipy/spatial/distance.py
Function: cdist at line 2060

Line #      Hits         Time  Per Hit   % Time  Line Contents
  2060                                           def cdist(XA, XB, metric='euclidean', *args, **kwargs):
  2061                                               """
  2062                                               Computes distance between each pair of the two collections of inputs.
  2063                                           
  2064                                               See Notes for common calling conventions.
  2065                                           
  2066                                               Parameters
  2067                                               ----------
  2068                                               XA : ndarray
  2069                                                   An :math:`m_A` by :math:`n` array of :math:`m_A`

In [30]:
%timeit
np.linalg.norm(X-Y)

ValueError: operands could not be broadcast together with shapes (777,100) (500,100) 

### Another one

In [31]:
from sklearn.metrics.pairwise import euclidean_distances as sklearn_euclidean_distances

In [32]:
D_sklearn = sklearn_euclidean_distances(X, Y)
np.abs(D_sklearn - D_scipy).sum()

3.789295988099184e-09

In [33]:
%%timeit 
sklearn_euclidean_distances(X, Y)

6.71 ms ± 186 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [35]:
profile_print(sklearn_euclidean_distances, X, Y)

Timer unit: 1e-06 s

Total time: 0.003574 s
File: /home/dmitry/anaconda3/lib/python3.6/site-packages/sklearn/metrics/pairwise.py
Function: euclidean_distances at line 163

Line #      Hits         Time  Per Hit   % Time  Line Contents
   163                                           def euclidean_distances(X, Y=None, Y_norm_squared=None, squared=False,
   164                                                                   X_norm_squared=None):
   165                                               """
   166                                               Considering the rows of X (and Y=X) as vectors, compute the
   167                                               distance matrix between each pair of vectors.
   168                                           
   169                                               For efficiency reasons, the euclidean distance between a pair of row
   170                                               vector x and y is computed as::
   171                  

### Numba

In [34]:
from numba import jit

In [35]:
@jit(nopython=True)
def vector_distance(x, y):
    dist = ((x - y) ** 2).sum()
    return np.sqrt(dist)

@jit(nopython=True)
def numba_mdist(X, Y):
    D = np.zeros((X.shape[0], Y.shape[0]))
    for i in range(X.shape[0]):
        for j in range(Y.shape[0]):
            D[i, j] = vector_distance(X[i, :], Y[j, :])
    return D

In [36]:
D_numba = numba_mdist(X, Y)
np.abs(D_numba - D_naive).sum()

3.3919462794074207e-09

In [37]:
%%timeit 
numba_mdist(X, Y) # x20

185 ms ± 1.89 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [35]:
# useless
profile_print(numba_mdist, X, Y)

Timer unit: 1e-06 s

Total time: 0 s
File: <ipython-input-32-b96f5878f15c>
Function: numba_mdist at line 6

Line #      Hits         Time  Per Hit   % Time  Line Contents
     6                                           @jit(nopython=True)
     7                                           def numba_mdist(X, Y):
     8                                               D = np.zeros((X.shape[0], Y.shape[0]))
     9                                               for i in range(X.shape[0]):
    10                                                   for j in range(Y.shape[0]):
    11                                                       D[i, j] = vector_distance(X[i, :], Y[j, :])
    12                                               return D



### Broadcast Numba

In [38]:
@jit(nopython=True)
def broadcast_numba_dist(X, Y):
    D = np.zeros((X.shape[0], Y.shape[0]))
    for j in range(Y.shape[0]):
        dist = ((X - Y[j, :]) ** 2).sum(axis=1)
        D[:, j] = np.sqrt(dist)
    return D

In [39]:
D_br_numba = broadcast_numba_dist(X, Y)
np.abs(D_br_numba - D_naive).sum()

3.3919462794074207e-09

In [40]:
%%timeit 
broadcast_numba_dist(X, Y)

97.1 ms ± 718 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


### Numpy numba 

In [41]:
@jit(nopython=True)
def numpy_numba_dist(X, Y):
    x_dist = (X ** 2).sum(axis=1)
    y_dist = (Y ** 2).sum(axis=1)
#     xy_dist = X.dot(Y.T)
#   !!!!!!!!!!!!!!!!!!!!!!!!!!!
    xy_dist = np.dot(X, Y.T)
    dist = - 2 * xy_dist + y_dist + x_dist.reshape(-1, 1)
    return np.sqrt(dist)

In [42]:
D_np_numba = numpy_numba_dist(X, Y)
np.abs(D_np_numba - D_naive).sum()

2.869199988708715e-09

In [43]:
%%timeit 
numpy_numba_dist(X, Y)

7.67 ms ± 249 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


### Cython

In [45]:
%%cython
import numpy as np
cimport numpy as np

def vector_distance(x, y):
    dist = ((x - y) ** 2).sum()
    return np.sqrt(dist)


def cython_mdist(X, Y):
    D = np.zeros((X.shape[0], Y.shape[0]))
    for i in range(X.shape[0]):
        for j in range(Y.shape[0]):
            D[i, j] = vector_distance(X[i, :], Y[j, :])
    return D

In [46]:
D_cython = cython_mdist(X, Y)
np.abs(D_cython - D_naive).sum()

0.0

In [47]:
%%timeit 
cython_mdist(X, Y)

2.69 s ± 72.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [46]:
# useless
profile_print(cython_mdist, X, Y)

/Users/meudon/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: UserWarning: Could not extract a code object for the object <built-in function cython_mdist>
  This is separate from the ipykernel package so we can avoid doing imports until


Timer unit: 1e-06 s



### Cython annotations

In [48]:
%%cython -a
import numpy as np
cimport numpy as np

cdef vector_distance(x, y):
    dist = ((x - y) ** 2).sum()
    return np.sqrt(dist)


cdef cython_mdist(X, Y):
    D = np.zeros((X.shape[0], Y.shape[0]))
    for i in range(X.shape[0]):
        for j in range(Y.shape[0]):
            D[i, j] = vector_distance(X[i, :], Y[j, :])
    return D

In [49]:
%%cython -a
import numpy as np
cimport numpy as np

cdef vector_distance(x, y):
    dist = ((x - y) ** 2).sum()
    return np.sqrt(dist)


cdef cython_mdist(np.ndarray[np.float64_t, ndim=2] X, 
                  np.ndarray[np.float64_t, ndim=2] Y):
    cdef np.ndarray[np.float64_t, ndim=2] D;
    D = np.zeros((X.shape[0], Y.shape[0]))
    for i in range(X.shape[0]):
        for j in range(Y.shape[0]):
            D[i, j] = vector_distance(X[i, :], Y[j, :])
    return D

In [50]:
D_cython = cython_mdist(X, Y)
np.abs(D_cython - D_naive).sum()

0.0

In [51]:
%%timeit 
cython_mdist(X, Y)

2.69 s ± 100 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [52]:
%%cython -a
import numpy as np
cimport numpy as np
cimport cython

from libc.math cimport sqrt


cpdef cython_mdist(np.ndarray[np.float64_t, ndim=2] X, 
                  np.ndarray[np.float64_t, ndim=2] Y):
    cdef np.ndarray[np.float64_t, ndim=2] D;
    D = np.zeros((X.shape[0], Y.shape[0]), dtype=np.float64)
    cdef np.float64_t dist = 0.0;
    for i in range(X.shape[0]):
        for j in range(Y.shape[0]):
            dist = 0.0
            for d in range(X.shape[1]):
                dist += (X[i, d] - Y[j, d]) ** 2
            D[i, j] = sqrt(dist)
    return D

In [53]:
D_cython = cython_mdist(X, Y)
np.abs(D_cython - D_naive).sum()

3.3919462794074207e-09

In [54]:
%%timeit 
cython_mdist(X, Y)

116 ms ± 2.87 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [58]:
%%cython -a
import numpy as np
cimport numpy as np
cimport cython

from libc.math cimport sqrt


@cython.boundscheck(False)
@cython.wraparound(False)
cpdef cython_mdist(np.ndarray[np.float64_t, ndim=2] X, 
                  np.ndarray[np.float64_t, ndim=2] Y):
    cdef np.ndarray[np.float64_t, ndim=2] D;
    D = np.zeros((X.shape[0], Y.shape[0]), dtype=np.float64)
    cdef np.float64_t dist = 0.0;
    for i in range(X.shape[0]):
        for j in range(Y.shape[0]):
            dist = 0.0
            for d in range(X.shape[1]):
                dist += (X[i, d] - Y[j, d]) ** 2
            D[i+1, j] = sqrt(dist)
    return D

In [59]:
D_cython = cython_mdist(X, Y)
np.abs(D_cython - D_naive).sum()

IndexError: index 777 is out of bounds for axis 0 with size 777

In [57]:
%%timeit 
cython_mdist(X, Y)

42.1 ms ± 1.15 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
